In [ ]:
from htmd import *
htmd.config(viewer='VMD')
def analysis(sims,boot=0.8,clusters=1000,merge=5):
    metr = Metric(sims)
    metr.projection(MetricDistance('protein and name CA', 'resname MOL and noh', metric='contacts'))
    data = metr.project()
    data.fstep = 0.1
    data.plotTrajSizes()
    print('\n\n\n\n\n')
    data.dropTraj()
    tica = TICA(data, 10)
    dataTica = tica.project(3)
    dataBoot = dataTica.bootstrap(0.8)
    dataBoot.cluster(MiniBatchKMeans(n_clusters=1000), mergesmall=5) #try with dataTica instead of dataBoot
    model = Model(dataBoot) #try with dataTica
    model.plotTimescales() 
    print('\n\n\n\n\n')
    nframes=input('At which time do time scales converge?')
    ntimescales=input('How many different time scales do you see?')
    model.markovModel(int(nframes)*10, int(ntimescales)) 
    mols = model.getStates()

    model.viewStates(ligand='resname MOL and noh')
    kin = Kinetics(model, temperature=298, concentration=0.0037)
    goodmacros=dict()
    mols = model.getStates()
    for i in range(len(mols)):
        for j in range(len(mols)):
            r = kin.getRates(i,j)
            if r.g0eq < -0.2:
                try:
                    goodmacros[i].append(j)
                except:
                    goodmacros[i]=[j]
    
    curr_max_len=0
    for keys in goodmacros:
        if len(goodmacros[keys])>curr_max_len:
            curr_max_len=len(goodmacros[keys])
            thekey=keys

    retlist=list()
    for sinks in goodmacros[thekey]:
        retlist.append(mols[sinks])
    kin.plotRates(rates=('g0eq'))
    print("---")
    kin.plotFluxPathways()    
    return retlist

In [ ]:
%pylab inline
all_molecules=list()
all_dir=glob('./*_sampling/filtered/')
for directory in all_dir:
    print("\n\n\n========== Current ligand directory:",directory," ==========\n")
    try:
        print(glob(directory+'*/'), glob(directory+'/filtered.pdb'))
        sims = simlist(glob(directory+'*/'), glob(directory+'/filtered.pdb'))
        all_molecules.append(analysis(sims,boot=0.8,clusters=1000,merge=5)) 
    except:
        print('\n########################################################## \n\n Trajectories are to small. Starting analysis for next folder.\n\n ########################################################## \n')
        continue

In [ ]:
print(all_molecules)